In [75]:
import json

relations = []
for line in open('/Users/alexmichailidis/gg-extraction-2022/er_three.jsonl', 'r'):
    relations.append(json.loads(line))

In [77]:
relations.pop(149) #error
relations.pop(91) #error

{'text': '"1. Στρατηγικός σκοπός της Γενικής Διεύθυνσης είναι ο προγραμματισμός και η εφαρμογή πολιτικών και δράσεων ενδυνάμωσης της ισόρροπης περιφερειακής ανάπτυξης της Μακεδονίας και Θράκης, προστασίας και ανάδειξης της φυσικής, αρχιτεκτονικής και πολιτιστικής κληρονομιάς, προώθησης της συνεργασίας με τις χώρες της Βαλκανικής και του Εύξεινου Πόντου σε θέματα περιβάλλοντος, έργων υποδομής, πολιτιστικής κληρονομιάς, τουριστικής ανάπτυξης και απόδημου ελληνισμού καθώς και ο σχεδιασμός, συντονισμός, διαχείριση και εποπτεία όλων των θεμάτων που άπτονται της διοικητικής υποστήριξης, της ηλεκτρονικής διακυβέρνησης και της οικονομικής λειτουργίας του Φορέα και των εποπτευόμενων από εκείνον φορέων, σύμφωνα με τις αρχές της ποιότητας και αποδοτικότητας και τους κανόνες της χρηστής δημοσιονομικής διαχείρισης και ευθύνης. Επιπλέον, περιλαμβάνονται η συνεργασία και διαρκής ενημέρωση της πολιτικής ηγεσίας σχετικά με τους δημοσιονομικούς περιορισμούς ή/και τις δυνατότητες ανάπτυξης, τροποποίησης 

{'text': '"1. Στρατηγικός σκοπός της Γενικής Διεύθυνσης είναι ο προγραμματισμός και η εφαρμογή πολιτικών και δράσεων ενδυνάμωσης της ισόρροπης περιφερειακής ανάπτυξης της Μακεδονίας και Θράκης, προστασίας και ανάδειξης της φυσικής, αρχιτεκτονικής και πολιτιστικής κληρονομιάς, προώθησης της συνεργασίας με τις χώρες της Βαλκανικής και του Εύξεινου Πόντου σε θέματα περιβάλλοντος, έργων υποδομής, πολιτιστικής κληρονομιάς, τουριστικής ανάπτυξης και απόδημου ελληνισμού καθώς και ο σχεδιασμός, συντονισμός, διαχείριση και εποπτεία όλων των θεμάτων που άπτονται της διοικητικής υποστήριξης, της ηλεκτρονικής διακυβέρνησης και της οικονομικής λειτουργίας του Φορέα και των εποπτευόμενων από εκείνον φορέων, σύμφωνα με τις αρχές της ποιότητας και αποδοτικότητας και τους κανόνες της χρηστής δημοσιονομικής διαχείρισης και ευθύνης. Επιπλέον, περιλαμβάνονται η συνεργασία και διαρκής ενημέρωση της πολιτικής ηγεσίας σχετικά με τους δημοσιονομικούς περιορισμούς ή/και τις δυνατότητες ανάπτυξης, τροποποίησης 

In [79]:
type(relations[1]["relations"][1]["label"])

str

In [80]:
#there was a problem in some cases during the export and in the spans (entities) it didn't get the text and the tye of entity. 
#This code makes the corrections

for r in range(len(relations)):

    for item in relations[r]["spans"]:
        if len(item) < 7:

            words=[]
            sentence=[]
            for i in range(len(relations[r]["tokens"])):
                if relations[r]["tokens"][i]["start"] == item["start"]:
                    for j in range(i,len(relations[r]["tokens"])):
                        words.append(relations[r]["tokens"][j])
            
            for word in words:
                sentence.append(word["text"])
                if item["end"]== word["end"]:
                    text=' '.join(word for word in sentence)
                    item["text"]= text
                    item["type"]= 'span'
                    break

In [81]:
#Transform based on the input desired by sample

desired_key_list = ["em1Text", "em2Text", "label"]
desired_order_list = ["start", "label", "text"]


for i in range(len(relations)-1,-1,-1):
    if relations[i]["answer"] == 'accept':
        
        #first part remove unnecessary values
        relations[i].pop("_input_hash")
        relations[i].pop("_task_hash")
        relations[i].pop("_is_binary")
        #relations[i].pop("meta")
        relations[i].pop("tokens")
        relations[i].pop("_view_id")
        relations[i].pop("_timestamp")
        relations[i].pop("answer")

        #second part transform relations
        for lst in range(len(relations[i]["relations"])):
            
            relations[i]["relations"][lst].pop('head')
            relations[i]["relations"][lst].pop('child')
            relations[i]["relations"][lst].pop('color')
            
            for item in relations[i]["spans"]:
                if relations[i]["relations"][lst]["head_span"]["start"] == item["start"]:
                    relations[i]["relations"][lst]["em2Text"] = item["text"]

                if relations[i]["relations"][lst]["child_span"]["start"] == item["start"]:
                    relations[i]["relations"][lst]["em1Text"] = item["text"]
                    

            relations[i]["relations"][lst].pop('head_span')
            relations[i]["relations"][lst].pop('child_span')
            label = relations[i]["relations"][lst]["label"]
            relations[i]["relations"][lst]["label"] ='Org/Org/{:s}'.format(label)
            relations[i]["relations"][lst] = {k: relations[i]["relations"][lst][k] for k in desired_key_list}

        #third part transform spans
        j=1
        for item in relations[i]["spans"]:
                
            item.pop("token_start")
            item.pop("token_end")
            item.pop("end")
            item.pop("type")
            item["start"] = j
            j+=1
            
            item = {k: item[k] for k in desired_order_list}
            
            
        # finally rename keys
        relations[i]["sentText"] = relations[i].pop("text")
        relations[i]["entityMentions"] = relations[i].pop("spans")
        relations[i]["relationMentions"] = relations[i].pop("relations")
    
    else:
        relations.pop(i)

In [83]:
#have a look on transformed data
relations[0]

{'sentText': '"Το Υπουργείο Εργασίας, Κοινωνικής Ασφάλισης και Κοινωνικής Αλληλεγγύης διαρθρώνεται, ως εξής: 1. α. Πολιτικό Γραφείο Υπουργού. β. Πολιτικά Γραφεία Αναπληρωτών Υπουργών. γ. Πολιτικό Γραφείο Υφυπουργού. δ. Γραφείο Γενικού Γραμματέα Υπουργείου Εργασίας,Κοινωνικής Ασφάλισης και Κοινωνικής Αλληλεγγύης. ε. Γραφείο Γενικού Γραμματέα Γενικής Γραμματείας Κοινωνικής Αλληλεγγύης. στ. Γραφείο Γενικού Γραμματέα Γενικής Γραμματείας Κοινωνικών Ασφαλίσεων. ζ. Γραφείο Γενικού Γραμματέα Γενικής Γραμματείας Διαχείρισης Κοινοτικών και Άλλων Πόρων. η. Γραφείο Ειδικού Γραμματέα Σώματος Επιθεώρησης Εργασίας. θ. Γραφείο Ειδικού Γραμματέα Κοινωνικής και Αλληλέγγυας Οικονομίας. ι. Γραφείο Ειδικού Γραμματέα Κοινωνικής Ένταξης των Ρομά."',
 'entityMentions': [{'text': 'Υπουργείο Εργασίας, Κοινωνικής Ασφάλισης και Κοινωνικής Αλληλεγγύης',
   'start': 1,
   'label': 'ORG'},
  {'text': 'Πολιτικό Γραφείο Υπουργού', 'start': 2, 'label': 'ORG'},
  {'text': 'Πολιτικά Γραφεία Αναπληρωτών Υπουργών',
   'sta

In [85]:
#save each in a row

filename = '/Users/alexmichailidis/gg-extraction-2022/er-three.json'

with open(filename, "w") as fp: 
    for my_dict in relations:
        if (my_dict!=relations[-1]):
            fp.write(json.dumps(my_dict, ensure_ascii=False)+'\n')
        else:
            fp.write(json.dumps(my_dict, ensure_ascii=False)) 

In [ ]:
#alternative way not in a row
filename = '/Users/alexmichailidis/gg-extraction-2022/data/er_test.json'

with open(filename, "w") as outfile:  # also, tried mode="rb"
    json.dump(relations, outfile, ensure_ascii=False, indent=2)